# Lab 3: Grammar writing
### COSC 426: Fall 2025, Colgate University

Use this notebook to answer the questions in `Lab4.md`. Make sure to include in this notebook all the tests you run to ensure that at each stage your n-gram model implementation is correct. Please use **markdown** chunks to answer any non-code questions. 

## Part 1

Answer questions for each of the subparts here. Add as many markdown chunks as needed under each sub-part.

### Part 1.1

C(the, panda) = 1
C(the) = 3
P(panda | the) = 1/3

C(a, sandwich) = 1
C(a) = 2
P(sandwich | a) = 1/2

C(in, the) = 0
C(in) = 2
P(the | in) = 0/2

C(panda, eats) = 1
C(panda) = 3
P(eat | panda) = 1/3

### Part 1.2

C([BOS], the) = 2
C([BOS]) = 3
P(the | [BOS]) = 2/3

C([UNK], panda) = 1
C([UNK]) = 5
P(panda|[UNK]) = 1/5

C(likes, [UNK]) = 1
C(likes) = 2
P([UNk] | likes) = 1/2

C([UNK], peacefully) = 0
C([UNK]) = 5
P(peacefully | [UNK]) = 0

C([UNK], [UNK]) = 2
C([UNK]) = 5
P([UNK] | [UNK]) = 2/5

C(panda, [EOS]) = 1
C(panda) = 3
P([EOS]|panda) = 1/3

C(likes, sandwich) = 0
C(likes) = 3
P(sandwich | likes) = 0

It assigns high probality for words that are more likely to begin a sentence. It also helps to differentiate between sentences; to know when a sentence ends and when it begins.

Words that are not present in the vocan with out the tag will have a prob of 0, so with the UNK tag, it helps you quantify how many words are not present in the vocab. This helps to assign somewhat better probabilities to contexts where we have unknown words.  


### Part 1.3

Add 1 smoothing add one to both numerator and the count vacab size.
P(panda | the) = 1 + 1/3 + 10 = 2/13

P(sandwich | a) = 1+1 / 2 + 10 = 1/6

P(the | in) = 0 + 1 / 2 + 10 = 1/12

P(eat | panda) = 1+1 / 3+10 = 2/13

P(the | [BOS]) = 2 + 1 / 3 + 10 = 3/13

P([UNK] | [UNK]) = 2+1 / 5+10 = 1/5

P([EOS]|panda) = 1+1 / 3+10 = 2/13

1. There might be words sequence that appear in the vocabulary but not in the corpus; if there is no smoothing their count/probability will be zero. By smoothing, we can account for such combination of words and assign some probability to their occurence. If we are ever evaluating a model, we want the model to know that the those combination of words are possible, and should not assign zero  probability to those. And we achieve this by smoothing. 

2. It reduces the probability of the bigrams. It distributes some of the known probailities to some unknown probabilities. 

3. add - 0.1; this is because it reduces the unsmoothed bigram probability estimates the least. Compared with the unsmoothed probabilities, the add-0.1 probabilities are closest to the initial probabailities without smoothing.  

4. Smaller values of k will be better. This will redistribute very small portion of our probability mask to unknown texts. This is preferred because our training text is very similar to our text text. 



## Part 2

Use this part to test your code. Feel free to add any additional code chunks. Note, once you make changes to `Lab4.py` you might have to restart the kernel for the changes to be reflected here!

In [1]:
## DO NOT CHANGE THIS CHUNK
import Lab4
import doctest

doctest.testmod(Lab4, verbose=True) ## runs the doctest

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ernestclottey/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


1.0
0.6666666666666666
0.0
0.0
Trying:
    TestBigramFreqs(getBigramFreqs(preprocess('data/test.txt', mark_ends=True), getVocab('data/glove_vocab.txt')))
Expecting:
    {1: 70, 2: 3}
ok
Trying:
    TestBigramFreqs(getBigramFreqs(preprocess('data/test.txt', mark_ends=True), getVocab('data/glove_vocab.txt')), print_non1=True)
Expecting:
    {2: [('kitten', 'had'), ('.', '[EOS]'), ('for', 'the')]}
ok
Trying:
    len(getVocab('data/glove_vocab.txt'))
Expecting:
    400003
ok
Trying:
    preprocess('data/test.txt', mark_ends=True)
Expecting:
    [['[BOS]', 'one', 'thing', 'was', 'certain', ',', 'that', 'the', '_white_', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', ':', '—it', 'was', 'the', 'black', 'kitten', '’', 's', 'fault', 'entirely', '.', '[EOS]'], ['[BOS]', 'for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', '(', 'and', 'bearing', 'it', 'pretty', 'well', ',', 

TestResults(failed=0, attempted=5)

In [2]:
## Correct probs to test your implementation of getBigramProb()
correct_probs_mle = {
        ('one', 'thing'): 1.0,
        ('kitten', 'had'): 0.6666666666666666,
        ('cat', 'had'): 0.0,
        ('had', 'had'): 0.25,
        ('on', 'the'): 0.0,
        ('held', 'a'): 0.0,
        ('zzzzzzz', 'the'): 0.0
    }

correct_probs_add1 = {
        ('one', 'thing'): 4.999950000499995e-06,
        ('kitten', 'had'): 7.499887501687475e-06,
        ('cat', 'had'): 2.4999750002499977e-06,
        ('had', 'had'): 4.999912501531223e-06,
        ('on', 'the'): 2.499981250140624e-06,
        ('held', 'a'): 2.499981250140624e-06,
        ('zzzzzzz', 'the'): 2.4999562507656116e-06
    }

In [14]:
vocab = Lab4.getVocab('data/glove_vocab.txt')

#train on through_the_looking_glass.txt
sentences = Lab4.preprocess('data/test.txt', mark_ends=True)

#this is dictionary for through_the_looking_glass file
bigram_freq = Lab4.getBigramFreqs(sentences, vocab)
word_freq = Lab4.get_word_freq_dict(sentences, vocab) #of my training data/text

In [16]:
#code to test getprob. 
for key in correct_probs_mle:
    print(Lab4.getBigramProb(key, "MLE", bigram_dict=bigram_freq, vocabs = vocab, dict_word_freq=word_freq) == correct_probs_mle[key])
    
for key in correct_probs_add1:
    # print(key)
    # print(correct_probs_add1[key])
    # print(Lab4.getBigramProb(key, "add-1", bigram_dict=bigram_freq, vocabs = vocab, dict_word_freq=word_freq))
   print(Lab4.getBigramProb(key, "add-1", bigram_dict=bigram_freq, vocabs = vocab, dict_word_freq=word_freq) == correct_probs_add1[key])

True
True
True
True
True
True
True
True
True
True
True
False
False
True


In [17]:
for key in correct_probs_add1:
    print(key)
    print(correct_probs_add1[key])
    print(Lab4.getBigramProb(key, "add-1", bigram_dict=bigram_freq, vocabs = vocab, dict_word_freq=word_freq))

('one', 'thing')
4.999950000499995e-06
4.999950000499995e-06
('kitten', 'had')
7.499887501687475e-06
7.499887501687475e-06
('cat', 'had')
2.4999750002499977e-06
2.4999750002499977e-06
('had', 'had')
4.999912501531223e-06
4.999912501531223e-06
('on', 'the')
2.499981250140624e-06
2.4999562507656116e-06
('held', 'a')
2.499981250140624e-06
2.4999562507656116e-06
('zzzzzzz', 'the')
2.4999562507656116e-06
2.4999562507656116e-06


The bigrams ('held', 'a') and ('on', 'the') not pass the tests i run on getBrigramProb. The values are teh same up to the fourth decimal place, and I am not particularly sure about what is causing that problem.

For ('on', 'the')
expected: 2.499981250140624e-06
got: 2.4999562507656116e-06

For ('held', 'a')
expected: 2.499981250140624e-06
got: 2.4999562507656116e-06

## Part 3

Use this part to answer questions in Part 3. Add as many code and markdown chunks as is helpful. **Remember, once you make changes to your `Lab4.py`, you need to restart the kernel and reimport the file**. 

In [11]:
#vocab is preloaded above
#vocab = Lab4.getVocab('data/glove_vocab.txt')

#train on through_the_looking_glass.txt
sentences = Lab4.preprocess('data/through_the_looking_glass.txt', mark_ends=False)

#this is dictionary for through_the_looking_glass file
bigram_freq = Lab4.getBigramFreqs(sentences, vocab)
word_freq = Lab4.get_word_freq_dict(sentences, vocab) #of my training data/text

N = Lab4.corpus_size(sentences)

#evaluating the model on through_the_looking_glass.txt
print(Lab4.evaluate_model(N, sentences, "add-1", vocab, bigram_freq, word_freq))


37436.64615981037


In [12]:
# evaluate on sherlock, change sentences to the corpus from sherlock text. 
# But maintain the bigram_freq dictionary from through_the_looking_glass
sentences_alice = Lab4.preprocess('data/alice_in_wonderland.txt', mark_ends=False)
N_alice = Lab4.corpus_size(sentences_alice)
#word_freq = Lab4.get_word_freq_dict(sentences_alice)

#evaluating the model on alice_in_wonderland.txt
print(Lab4.evaluate_model(N_alice, sentences_alice, "add-1", vocab, bigram_freq, word_freq))

55264.87572188816


In [13]:
sentences_sherlock = Lab4.preprocess('data/sherlock_holmes.txt', mark_ends=False)
N_sherlock = Lab4.corpus_size(sentences_sherlock)

#evaluating the model on sherlock_holmes.txt
print(Lab4.evaluate_model(N_sherlock, sentences_sherlock, "add-1", vocab, bigram_freq, word_freq))

87140.57425787832


Perplexity, which measures a model's surprise with a given text, varies depending on the content. "Through the Looking-Glass" resulted in the lowest perplexity after evaluation, which is expected since the model was trained on that specific text and is thus less surprised by it. The "sherlock.txt" file, being longer, had a higher perplexity. This is likely because the model encountered a greater number of new words, leading to increased surprisal. 

## Part 4 (optional)
Use this part to answer questions in Part 3. Add as many code and markdown chunks as is helpful.